https://www.analyticsvidhya.com/blog/2022/01/sentiment-analysis-with-lstm/ <br>
https://www.embedded-robotics.com/sentiment-analysis-using-lstm/

In [1]:
import os
import re
import keras
import math
import nltk
import collections
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
data = pd.read_csv('comments_word.csv')
data

,index,name,date,content,label
0,1,Lyka Garcia,8/8/2022 1:17,guard may baliw dito,hate
1,2,Eden Abalayan Avila,8/8/2022 1:44,So pag historian may kasama talgang tsismis???...,non-hate
2,3,Kaken Ocampo,8/8/2022 1:51,Pahuway!,non-hate
3,4,Natasha Cuyugan Roque,8/8/2022 2:46,Sobrang taba talaga ng utak ni direk!,non-hate
4,5,Arjay Santos,8/8/2022 2:47,Hay naku! E talaga naman nagreresearch ang mga...,hate
...,...,...,...,...,...
2651,2656,Che EC Olar,02/01/23 02:02:32,Di dapat sinusuportahan ang pelikula ng direkt...,hate
2652,2657,Romeo Borcelis,02/01/23 02:05:57,Anong klaseng director puro laman ng imburnal ...,hate
2653,2658,Jho Sie,02/01/23 02:06:03,Ito ba yung klase ng respetadong direktor kuno?,hate
2654,2659,Emjay Solomon,02/01/23 02:12:12,Olat ang dakilang sipsip...hahahaha,hate


In [3]:
def remove_html(text):
    cleantext = re.sub('<[^<]+?>', '', text)
    return cleantext
    
def remove_punct(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]'%re.escape(string.punctuation), '', text)
    text = re.sub('w*\d\w*', '', text)
    return text

def clean_text2(text):
    text = re.sub('[''"",,,]', '', text)
    text = re.sub('\n', '', text)
    return text

cleaned_html = lambda x:clean_html(x)
cleaned1 = lambda x:clean_text1(x)
cleaned2 = lambda x:clean_text2(x)

data['content'] = data['content'].apply(lambda x: remove_html(x))
data['content'] = data['content'].apply(lambda x: remove_punct(x))
data['content'] = data['content'].apply(lambda x: clean_text2(x))

In [4]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in data.content]
tokens

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Florence\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[['guard', 'may', 'baliw', 'dito'],
 ['so',
  'pag',
  'historian',
  'may',
  'kasama',
  'talgang',
  'tsismiskung',
  'ganun',
  'npaka',
  'tsismoso',
  'mong',
  'direktorhahahaahha'],
 ['pahuway'],
 ['sobrang', 'taba', 'talaga', 'ng', 'utak', 'ni', 'direk'],
 ['hay',
  'naku',
  'e',
  'talaga',
  'naman',
  'nagreresearch',
  'ang',
  'mga',
  'historian',
  'or',
  'lahat',
  'naman',
  'ng',
  'tao',
  'eh',
  'kapag',
  'gustong',
  'malaman',
  'ang',
  'katotohanan',
  'kinakalap',
  'ang',
  'impormasyon',
  'kahit',
  'saan',
  'pagtataka',
  'ko',
  'lang',
  'kung',
  'bakit',
  'hindi',
  'daw',
  'profession',
  'ang',
  'pagiging',
  'historian',
  'ng',
  'mga',
  'taong',
  'kinakalap',
  'ang',
  'mga',
  'pangyayari',
  'sa',
  'mga',
  'nakaraang',
  'siglo',
  'grabe',
  'siya',
  'so',
  'ano',
  'pala',
  'ang',
  'tawag',
  'sa',
  'profession',
  'ng',
  'mga',
  'taong',
  'nag',
  'aral',
  'ng',
  'historyabhistory',
  'hay',
  'naku',
  'sir',
  'magdah

In [5]:
!pip install advertools
import advertools as adv
adv.stopwords.keys()
print(sorted(adv.stopwords['tagalog'])[:147])

['akin', 'aking', 'ako', 'alin', 'am', 'amin', 'aming', 'ang', 'ano', 'anumang', 'apat', 'at', 'atin', 'ating', 'ay', 'bababa', 'bago', 'bakit', 'bawat', 'bilang', 'dahil', 'dalawa', 'dapat', 'din', 'dito', 'doon', 'gagawin', 'gayunman', 'ginagawa', 'ginawa', 'ginawang', 'gumawa', 'gusto', 'habang', 'hanggang', 'hindi', 'huwag', 'iba', 'ibaba', 'ibabaw', 'ibig', 'ikaw', 'ilagay', 'ilalim', 'ilan', 'inyong', 'isa', 'isang', 'itaas', 'ito', 'iyo', 'iyon', 'iyong', 'ka', 'kahit', 'kailangan', 'kailanman', 'kami', 'kanila', 'kanilang', 'kanino', 'kanya', 'kanyang', 'kapag', 'kapwa', 'karamihan', 'katiyakan', 'katulad', 'kaya', 'kaysa', 'ko', 'kong', 'kulang', 'kumuha', 'kung', 'laban', 'lahat', 'lamang', 'likod', 'lima', 'maaari', 'maaaring', 'maging', 'mahusay', 'makita', 'marami', 'marapat', 'masyado', 'may', 'mayroon', 'mga', 'minsan', 'mismo', 'mula', 'muli', 'na', 'nabanggit', 'naging', 'nagkaroon', 'nais', 'nakita', 'namin', 'napaka', 'narito', 'nasaan', 'ng', 'ngayon', 'ni', 'nila',

In [6]:
stoplist = adv.stopwords['tagalog']
stoplist

{'akin',
 'aking',
 'ako',
 'alin',
 'am',
 'amin',
 'aming',
 'ang',
 'ano',
 'anumang',
 'apat',
 'at',
 'atin',
 'ating',
 'ay',
 'bababa',
 'bago',
 'bakit',
 'bawat',
 'bilang',
 'dahil',
 'dalawa',
 'dapat',
 'din',
 'dito',
 'doon',
 'gagawin',
 'gayunman',
 'ginagawa',
 'ginawa',
 'ginawang',
 'gumawa',
 'gusto',
 'habang',
 'hanggang',
 'hindi',
 'huwag',
 'iba',
 'ibaba',
 'ibabaw',
 'ibig',
 'ikaw',
 'ilagay',
 'ilalim',
 'ilan',
 'inyong',
 'isa',
 'isang',
 'itaas',
 'ito',
 'iyo',
 'iyon',
 'iyong',
 'ka',
 'kahit',
 'kailangan',
 'kailanman',
 'kami',
 'kanila',
 'kanilang',
 'kanino',
 'kanya',
 'kanyang',
 'kapag',
 'kapwa',
 'karamihan',
 'katiyakan',
 'katulad',
 'kaya',
 'kaysa',
 'ko',
 'kong',
 'kulang',
 'kumuha',
 'kung',
 'laban',
 'lahat',
 'lamang',
 'likod',
 'lima',
 'maaari',
 'maaaring',
 'maging',
 'mahusay',
 'makita',
 'marami',
 'marapat',
 'masyado',
 'may',
 'mayroon',
 'mga',
 'minsan',
 'mismo',
 'mula',
 'muli',
 'na',
 'nabanggit',
 'naging',
 '

In [7]:
data['content'] = data['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stoplist)]))

In [8]:
data

,index,name,date,content,label
0,1,Lyka Garcia,8/8/2022 1:17,guard baliw,hate
1,2,Eden Abalayan Avila,8/8/2022 1:44,so pag historian kasama talgang tsismiskung ga...,non-hate
2,3,Kaken Ocampo,8/8/2022 1:51,pahuway,non-hate
3,4,Natasha Cuyugan Roque,8/8/2022 2:46,sobrang taba talaga utak direk,non-hate
4,5,Arjay Santos,8/8/2022 2:47,hay naku e talaga naman nagreresearch historia...,hate
...,...,...,...,...,...
2651,2656,Che EC Olar,02/01/23 02:02:32,di sinusuportahan pelikula direktor marumi lum...,hate
2652,2657,Romeo Borcelis,02/01/23 02:05:57,anong klaseng director puro laman imburnal lam...,hate
2653,2658,Jho Sie,02/01/23 02:06:03,ba yung klase respetadong direktor kuno,hate
2654,2659,Emjay Solomon,02/01/23 02:12:12,olat dakilang sipsiphahahaha,hate


In [9]:
s = 0.0
for i in data['content']:
    word_list = i.split()
    s = s + len(word_list)
print("Average length of each comment : ",s/data.shape[0])
pos = 0
for i in range(data.shape[0]):
    if data.iloc[i]['label'] == 'hate':
        pos = pos + 1
neg = data.shape[0]-pos
print("Percentage of reviews with hate sentiment is "+str(pos/data.shape[0]*100)+"%")
print("Percentage of reviews with non-hate sentiment is "+str(neg/data.shape[0]*100)+"%")

Average length of each comment :  8.726280120481928
Percentage of reviews with hate sentiment is 49.13403614457831%
Percentage of reviews with non-hate sentiment is 50.86596385542169%


In [10]:
reviews = data['content'].values
labels = data['label'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [11]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, test_size=0.20)
print("Training Data Input Shape: ", train_sentences.shape)
print("Training Data Output Shape: ", train_labels.shape)
print("Testing Data Input Shape: ", test_sentences.shape)
print("Testing Data Output Shape: ", test_labels.shape)

Training Data Input Shape:  (2124,)
Training Data Output Shape:  (2124,)
Testing Data Input Shape:  (532,)
Testing Data Output Shape:  (532,)


In [12]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = ''
embedding_dim = 100
max_length = 200 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'

# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
print("Number of Documents: ", tokenizer.document_count)
print("Number of Words: ", tokenizer.num_words)

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

Number of Documents:  2124
Number of Words:  3000


In [13]:
tokenizer.word_counts

OrderedDict([('flop', 3),
             ('yan', 292),
             ('sure', 6),
             ('bwahahahahahahahaha', 1),
             ('simple', 5),
             ('kopal', 2),
             ('haha', 24),
             ('hala', 13),
             ('pangit', 4),
             ('naman', 188),
             ('ganon', 8),
             ('talaga', 110),
             ('mahal', 36),
             ('mo', 298),
             ('taong', 21),
             ('mahalaga', 4),
             ('buhay', 47),
             ('masakit', 9),
             ('maiwan', 1),
             ('lalo', 27),
             ('nanay', 12),
             ('bat', 29),
             ('di', 141),
             ('hire', 1),
             ('bagong', 9),
             ('editor', 3),
             ('d', 38),
             ('gago', 172),
             ('hahahahahahahaha', 2),
             ('laglaggaling', 1),
             ('talo', 7),
             ('is', 31),
             ('real', 3),
             ('tao', 54),
             ('tau', 3),
             ('magp

In [14]:
tokenizer.word_docs

defaultdict(int,
            {'flop': 3,
             'yan': 240,
             'bwahahahahahahahaha': 1,
             'sure': 6,
             'haha': 23,
             'kopal': 2,
             'simple': 5,
             'naman': 166,
             'hala': 13,
             'pangit': 4,
             'maiwan': 1,
             'masakit': 9,
             'ganon': 8,
             'talaga': 103,
             'lalo': 26,
             'nanay': 10,
             'mahalaga': 4,
             'mahal': 30,
             'taong': 20,
             'mo': 226,
             'buhay': 40,
             'bat': 28,
             'editor': 3,
             'd': 34,
             'bagong': 8,
             'hire': 1,
             'di': 128,
             'gago': 167,
             'hahahahahahahaha': 2,
             'university': 2,
             'magparami': 1,
             'talo': 7,
             'real': 3,
             'tau': 3,
             'tao': 51,
             'good': 14,
             'is': 28,
             'manmer

In [15]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='sigmoid'),
    keras.layers.Dense(1, activation='sigmoid')
])
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          300000    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              84480     
 l)                                                              
                                                                 
 dense (Dense)               (None, 24)                3096      
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [44]:
num_epochs = 10
history = model.fit(train_padded, train_labels, batch_size = 16,
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/10
120/120 [==============================] - 8s 65ms/step - loss: 0.0221 - accuracy: 0.9880 - val_loss: 1.3341 - val_accuracy: 0.7230
Epoch 2/10
120/120 [==============================] - 8s 68ms/step - loss: 0.0229 - accuracy: 0.9874 - val_loss: 1.3636 - val_accuracy: 0.7277
Epoch 3/10
120/120 [==============================] - 8s 63ms/step - loss: 0.0223 - accuracy: 0.9874 - val_loss: 1.3620 - val_accuracy: 0.7183
Epoch 4/10
120/120 [==============================] - 9s 75ms/step - loss: 0.0220 - accuracy: 0.9895 - val_loss: 1.3464 - val_accuracy: 0.7277
Epoch 5/10
120/120 [==============================] - 9s 71ms/step - loss: 0.0212 - accuracy: 0.9885 - val_loss: 1.3775 - val_accuracy: 0.7324
Epoch 6/10
120/120 [==============================] - 8s 70ms/step - loss: 0.0212 - accuracy: 0.9885 - val_loss: 1.4007 - val_accuracy: 0.7324
Epoch 7/10
120/120 [==============================] - 8s 66ms/step - loss: 0.0377 - accuracy: 0.9864 - val_loss: 1.3038 - val_accuracy: 0.7324

In [45]:
metrics_df = pd.DataFrame(history.history)
print(metrics_df)

       loss  accuracy  val_loss  val_accuracy
0  0.022127  0.987964  1.334100      0.723005
1  0.022872  0.987441  1.363569      0.727700
2  0.022276  0.987441  1.362029      0.718310
3  0.022030  0.989534  1.346400      0.727700
4  0.021221  0.988488  1.377489      0.732394
5  0.021245  0.988488  1.400654      0.732394
6  0.037703  0.986395  1.303751      0.732394
7  0.032519  0.982208  1.366605      0.737089
8  0.032284  0.983778  1.452001      0.713615
9  0.018285  0.990058  1.414564      0.727700


In [46]:
prediction = model.predict(test_padded)
# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(test_labels,pred_labels))

17/17 [==============================] - 1s 36ms/step
Accuracy of prediction on test set :  0.6710526315789473


In [47]:
print(classification_report(test_labels,pred_labels))

              precision    recall  f1-score   support

           0       0.66      0.67      0.67       261
           1       0.68      0.68      0.68       271

    accuracy                           0.67       532
   macro avg       0.67      0.67      0.67       532
weighted avg       0.67      0.67      0.67       532



In [48]:
# reviews on which we need to predict
sentence = ["napkawalang kwenta ni elizabeth, kung ako sa knya aalis nako", 
            "parng balilw, eh di nman maayos yung pagkakasabi niya", 
            "luh ampapanget ng gawa nila",
           "hahhaaha dpat di na nila pinkakita yon, sakit sa mata",
           "mas ok pa nung nakaraan kaysa ngayon eh ang gulo na nila",
           "parng gago kung makareklamo ung customer kanina",
           "sipain ko yon, di nga nagbabayad ng pinagkainan"]

# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get labels based on probability 1 if p>= 0.5 else 0
prediction = model.predict(padded)
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
for i in range(len(sentence)):
    print('Comment: ', sentence[i])
    if pred_labels[i] == 0:
        s = 'Hate'
    else:
        s = 'Non-hate'
    print("Predicted label : ",s)

1/1 [==============================] - 0s 38ms/step
Comment:  napkawalang kwenta ni elizabeth, kung ako sa knya aalis nako
Predicted label :  Hate
Comment:  parng balilw, eh di nman maayos yung pagkakasabi niya
Predicted label :  Hate
Comment:  luh ampapanget ng gawa nila
Predicted label :  Hate
Comment:  hahhaaha dpat di na nila pinkakita yon, sakit sa mata
Predicted label :  Non-hate
Comment:  mas ok pa nung nakaraan kaysa ngayon eh ang gulo na nila
Predicted label :  Non-hate
Comment:  parng gago kung makareklamo ung customer kanina
Predicted label :  Hate
Comment:  sipain ko yon, di nga nagbabayad ng pinagkainan
Predicted label :  Non-hate


In [49]:
# reviews on which we need to predict
sentence = ["saan ba makakakita nung ganung variation ng damit?", 
            "kinuha ko khapon sa may labahan ung bagong bili nyang dress eh", 
            "ang lupet nung design na ginawa ni aling myrna",
           "sarap sa mata tignan nung painting ni dyona, astig",
           "napakapromising ng acting niya, ramdam ko ung emosyon",
           "tama tama dapat lgi think positive tayo",
           "gusto ko ulit mkita  ung pagkakaluto sa humba"]

# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get labels based on probability 1 if p>= 0.5 else 0
prediction = model.predict(padded)
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
        
for i in range(len(sentence)):
    print('Comment: ', sentence[i])
    if pred_labels[i] == 0:
        s = 'Hate'
    else:
        s = 'Non-hate'
    print("Predicted label : ",s)

1/1 [==============================] - 0s 37ms/step
Comment:  saan ba makakakita nung ganung variation ng damit?
Predicted label :  Non-hate
Comment:  kinuha ko khapon sa may labahan ung bagong bili nyang dress eh
Predicted label :  Non-hate
Comment:  ang lupet nung design na ginawa ni aling myrna
Predicted label :  Non-hate
Comment:  sarap sa mata tignan nung painting ni dyona, astig
Predicted label :  Non-hate
Comment:  napakapromising ng acting niya, ramdam ko ung emosyon
Predicted label :  Non-hate
Comment:  tama tama dapat lgi think positive tayo
Predicted label :  Non-hate
Comment:  gusto ko ulit mkita  ung pagkakaluto sa humba
Predicted label :  Non-hate


In [50]:
# reviews on which we need to predict
sentence = ["luh ang bobo naman non", 
            "tanga amp", 
            "gago ka pala eh",
           "salamat naman ligtas ka",
           "ang ganda ng gawa mo",
           "wow ang galing mo naman po"]

# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get labels based on probability 1 if p>= 0.5 else 0
prediction = model.predict(padded)
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
        
for i in range(len(sentence)):
    print('Comment: ', sentence[i])
    if pred_labels[i] == 0:
        s = 'Hate'
    else:
        s = 'Non-hate'
    print("Predicted label : ",s)

1/1 [==============================] - 0s 79ms/step
Comment:  luh ang bobo naman non
Predicted label :  Hate
Comment:  tanga amp
Predicted label :  Hate
Comment:  gago ka pala eh
Predicted label :  Hate
Comment:  salamat naman ligtas ka
Predicted label :  Non-hate
Comment:  ang ganda ng gawa mo
Predicted label :  Non-hate
Comment:  wow ang galing mo naman po
Predicted label :  Non-hate
